# 8. Bildbearbeitung 2

In der Aufgabe 7 haben Sie zur Bearbeitung eines Bildes unterschiedliche Bildbearbeitungsfunktionen "manuell" implementiert.<br>
Python bietet jedoch eine Reihe von Bibliotheken, die diese Aufgabe enorm vereinfachen, die in dieser Aufgabe alternativ ausprobiert werden sollen.

Die Funktionen Funktionen zur Bildbearbeitung z.B. lassen sich mit den Mitteln von *numpy* und *scipy* viel einfacher realisieren.<br>
Diese Bibliotheken richten sich jedoch an Anwender, die über gewisse Kenntnisse in der Theorie der Bildbearbeitung verfügen und sind aus diesem Grund nicht ganz einfach anzuwenden. Die erforderlichen Funktionen und deren Parameter werden daher hier zur Verfügung gestellt.

## 8.1. Bilddatei lesen und speichern
### 8.1.1 Bilddatei lesen
<center>

![a7_dreifach.pgm](./figures/a7_abb_7_1_dreifach.png)<br>
Abbildung $7.1$.: Grauwertbild in der Datei `a7_dreifach.pgm`
</center>

Ausgangspunkt ist wieder die PGM-Bilddatei `a7_dreifach.pgm` (im Ordner `figures`).<br>
Auch in dieser Aufgabe werden zunächst zwei Funktionen benötigt, die Bilddateien im P2-PGM-Dateiformat lesen und schreiben können. Sie können die beiden Funktionen direkt aus Aufgabe 7 übernehmen und folgende Änderungen vornehmen:

Am Einlesen der Metadaten *Magic*, *Breite*, *Höhe* und *maximaler Grauwert* ändert sich nichts.<br>
Auch die erforderlichen Überprüfungen der Werte auf Gültigkeit bleiben erforderlich.<br>
Da die Prüfung der eingelesenen Werte eine wiederkehrende Aufgabe ist, wird die komplette Konvertierung mit Überprüfung in einer eigenen Funktion realisiert, die Sie im Mustercode finden. Der Kommentar dazu beschreibt die Anwendung.

Dazu gibt es in Python ein paar "nette" Erleichterungen.<br>
Mit
```
image['width'], image['height'] = 
   map(lambda s: toInt(s, min=1, caption="Höhe oder Breite"), file.readline().strip().split())
```
Können die Höhe und die Breite des Bildes "in einem Rutsch" in die beiden Variablen des *Dictonary* eingelesen werden. Hier die Erklärung der Funktion:<br>
* `file` sei der Name des Filehandles, mit dem die Datei geöffnet wurde.
* `file.readline().strip().split()` 
    * liest die komplette Zeile mit der Bildbreite und -höhe ein,
    * schneidet Leerzeichen am Anfang und Ende der Datei ab und
    * teilt mit `split` die beiden Werte in einzelne Strings auf.
* `map` verarbeitet jeden einzelnen der aufgeteilten Teilstrings, indem
    * dieser durch `lambda s` der Variablen s zugewiesen und mit
    * `lambda s: toInt(s, min=1, caption="Höhe oder Breite")` der Konvertierungsfunktion zur Konvertierung und Prüfung übergeben wird.<br>Zusätzlich wird festgelegt, dass die Werte größer oder gleich `1` sein müssen.<br>Für eine Mögliche Fehlermeldung wird `caption` mit einer geeigneten Bezeichnung der Werte belegt.
    * Die erfolgreich konvertierten Werte werden den Variablen `image['width'], image['height'] = ...` in der Reihenfolge von links nach rechts zugewiesen.

Die gleiche Funktionalität kann mit
```
pixels.extend(map(lambda s: toInt(s, 0, image['maxgray'], f"Zeile {rowIdx+1}"), line.split()))
```
dazu verwendet werden, alle Grauwerte einer Zeile (gelesen in die Variable `line`) auf einmal einer (eindimensionalen) Liste der Grauwerte mit dem Variablennamen `pixels` zuzufügen.
 
Die Bearbeitungsfunktionen der verwendeten Bibliotheken erfordern es, dass die Graustufenwerte der Pixel des Bildes nicht mehr als `int`-Werte in einer 2-dimensionalen "normalen" Python-Liste gespeichert sind, sondern in einem 2-dimensionalen `numpy.ndarray`.
* Lesen Sie die Graustufenwerte der Pixel zunächst in eine **<u>eindimensionale</u>**, "normale" Python-Liste (lokale Variable) ein.<br> Führend Sie dabei aller erforderlichen Prüfungen der Werte wie in Aufgabe 7 durch.
* Wenn Sie die Variable der Liste `pixels` genannt haben, übernehmen Sie anschließend die Graustufen-Werte mit folgendem Code in das *Dictionary* `image`, in dem Sie die Bilddaten speichern:
```
import numpy as np
...
image['pixels'] = np.array(pixels, dtype=np.uint8).reshape((image['height'], image['width']))

```
Die Datentypen in `numpy` (`dtype`) entsprechen den Datentypen, mit denen Prozessoren arbeiten und sind in den Bezeichnungen an die Datentypen angelehnt, die auch in der Programmiersprache C verwendet werden.<br>
`dtype=np.uint8` entspricht hier einem `unsigned char`, einer vorzeichenlosen 8-Bit Ganzzahl, die die Werte `0` bis `255` aufnehmen kann.

Die Funktion zum Einlesen gibt, wie in Aufgabe 7, das vollständig mit den Bilddaten des eingelesenen Bildes zurück.

### 8.1.2. Bilddatei speichern
Die Funktion zum Schreiben des mit den Bilddaten gefüllten *Dictionary* kann unverändert übernommen werden.

### 8.1.3 Anzeigen der Bilddatei
Zum Anzeigen der Bilddatei kann
```
from IPython.display import display
from PIL import Image
...
outfile="?.pgm"
with Image.open(outfile) as i:
    display(i)
```
verwendet werden.

In [ ]:
# Lösung 8.1. Bilddatei lesen und speichern
import os
import sys
from IPython.display import display
from PIL import Image
import numpy as np

################################################################################
# Konvertieren eines Strings in eine Ganzzahl.
# Konvertiert den übergeben String in eine Ganzzahl und übernimmt die Prüfung.
# Wirft ValueError wenn
# - Der String None ist
# - Der String ein Leerstring ("") ist
# - Der String nicht in einen int-Wert konvertiert werden kann.
# - Der int-Wert kleiner als min ist, falls min übergeben wird.
# - Der int-Wert größer als max ist, falls max übergeben wird.
# caption, falls übergeben,  wird für die Erstellung einer aussagefähigen 
# Fehlermeldung verwendet. caption sollte eine Beschreibung des Wertes enthalten,
# die es ermöglicht, den fehlerhaften Wert in der Datei wiederzufinden 
# (Höhe, Breite, Zeile, Spalte in der Datei, etc)
# token    Zu konvertierender String
# min      Minimalwert, falls erforderlich
# max      Maximalwert, falls erforderlich
# caption  Beschreibung des Wertes. Nicht erforderlich, aber wünschenswert.
def toInt(token: str, min:int=None, max:int=None, caption:str=None):
    result = None
    if (token is None):
        raise ValueError(f"{'' if caption is None else caption}{'' if caption is None else ': '}Ungültiger None-Wert!")
    token = token.strip()
    
    if (token == ""):
        raise ValueError(f"{'' if caption is None else caption}{'' if caption is None else ': '}Leerstring!")
    try:
        result = int(token)
    except ValueError:
        raise ValueError(f"{'' if caption is None else caption}{'' if caption is None else ': '}Keine gültige Ganzzahl!")
    if (min is not None and result < min):
        raise ValueError(f"{'' if caption is None else caption}{'' if caption is None else ': '}Wert ist kleiner {min}!")
    if (max is not None and result > max):
        raise ValueError(f"{'' if caption is None else caption}{'' if caption is None else ': '}Wert ist grösser {max}!")

    return result

################################################################################
# Lesen eines PGM Image-Files in ein Dictionary.
# fileName: Pfad zur PGM-Image-Datei
# return  : dict['magic']: Magic der PGM-Datei
#           dict['width']: Breite des Bildes
#           dict['height']: Höhe des Bildes
#           dict['maxgray'] Maximal vorkommender Grauwert.
#           dict['pixels']: numpy.ndarray mit zeilenweisen Graustufen des Bildes
################################################################################
def readPgmImageFile(fileName):
### BEGIN SOLUTION
    
    if (not os.path.exists(fileName)):
        raise FileNotFoundError(f"Datei {fileName} existiert nicht!")
    if (not os.path.isfile(fileName)):
        raise FileNotFoundError(f"Datei {fileName} existiert, ist aber keine Datei!")
    
    image = dict()
    image['pixels'] = None
    image['magic']= None
    image['width']= None
    image['height']= None
    image['maxgray']= None
    pixelCount=0
    with open(fileName, 'r', encoding='ascii') as file:
        image['magic'] = file.readline().strip()
        if image['magic'] != 'P2':
            raise ValueError("Das Bild ist kein PGM-Bild im P2 Format")
        image['width'], image['height'] = map(lambda s: toInt(s, min=1, caption="Höhe oder Breite"), file.readline().strip().split())
        image['maxgray'] = toInt(file.readline().strip(), 0, 255, "Maximaler Grauwert")
        pixels = []
        rowIdx = 0
        for line in file:
            pixels.extend(map(lambda s: toInt(s, 0, image['maxgray'], f"Zeile {rowIdx+1}"), line.split()))
            rowIdx += 1
        pixelCount = len(pixels)
        image['pixels'] = np.array(pixels, dtype=np.uint8).reshape((image['height'], image['width']))   
    # Bild auf Vollständigkeit prüfen
    if image['magic'] is None:
        raise Exception(f"{fileName}: Magic fehlt.")
    if image['width'] is None:
        raise Exception(f"{fileName}: Bildbreite fehlt.")
    if image['height'] is None:
        raise Exception(f"{fileName}: Bildhöhe fehlt.")
    if image['maxgray'] is None:
        raise Exception(f"{fileName}: Maximaler Grauwert fehlt.")
    if image['pixels'] is None:
        raise Exception(f"{fileName}: Keine Grauwerte für Bilder gefunden.")

    # Erwartete und gelesene Bilddimensionen vergleichen
    if (image['width'] * image['height'] != pixelCount):
        raise Exception(f"Erwarte {image['width']} * {image['height']} Pixel-Graustufenwerte, es sind aber {pixCount}")
### END SOLUTION        
    return image # Rückgabe des mit den Bilddaten gefüllten Dictionaries.

################################################################################
# Schreiben eines PGM Image Dictionaries in eine Datei.
# image:    image['magic']: Magic der PGM-Datei
#           image['width']: Breite des Bildes
#           image['height']: Höhe des Bildes
#           image['maxgray'] Maximal vorkommender Grauwert.
#           image['pixels']: numpy.ndarray mit zeilenweisen Graustufen des Bildes
# fileName: Pfad zur PGM-Image-Datei
################################################################################
def writePgmImageFile(image, fileName):
    try:
### BEGIN SOLUTION
        with open(fileName, 'w', encoding='ascii') as file:
            file.write(f"{image['magic']}\n")
            file.write(f"{image['width']}{image['height']:4d}\n")
            file.write(f"{image['maxgray']}\n")
            for row in image['pixels']:
                for pixel in row:
                    file.write(f"{pixel:4d} ")
                file.write("\n")
### END SOLUTION                
    except PermissionError as e:
        raise OSError(f"Rechte zum Schreiben der Datei {fileName} fehlen")
    except IOError as e:
        raise OSError(f"Fehler bei Schreiben der Datei {fileName}")
### BEGIN SOLUTION
    finally:
        file.close()
### END SOLUTION                
    
###########################################################
# Test
###########################################################
try:
### BEGIN SOLUTION
    infile="./figures/a7_dreifach.pgm"
    outfile="dreifach.out.pgm"

    image = readPgmImageFile(infile)
    writePgmImageFile(image, outfile)

    with Image.open(outfile) as i:
	    display(i)
 
except ValueError as e:
    sys.stderr.write("Fehler:\n")
    sys.stderr.write(str(e))
except OSError as e:
    sys.stderr.write("Fehler:\n")
    sys.stderr.write(str(e))
except Exception as e:
    sys.stderr.write("Unerwarteter Fehler:\n")
    sys.stderr.write(str(e))
    raise e
### END SOLUTION


## 8.2.  Grauwertbild bearbeiten

Nachdem die Daten eines PGM-Bildes in ein *Dictionary* eingelesen werden können, ist es an der Zeit, die Python-Bibliotheken für die Bildbearbeitung zu Verwenden.

Die gesamte Bearbeitung wird mit den Graustufenwerten im `numpy.ndarray` durchgeführt, das im Attribut<br>
`image['pixels']` des *Dictionary* gespeichert ist. Die sonstigen Metadaten werden nur für das Speichern des Bildes benötigt.

### 8.2.1 Invertierung
Wie in Aufgabe 7 muss zunächst ein weiteres *Dictionary* für das bearbeitete Bild angelegt und mit den Metadaten des Quellbildes belegt werden. Das Invertieren selbst ist ein Einzeiler:
```
result['pixels']  = image['maxgray'] - image['pixels']
```
"Gelesen" sieht es so aus, als würde ein 2-dimentionales `numpy.ndarray` von einem einzigen `int` Wert subtrahiert.<br>
Tatsächlich wird aber für jeden einzelnen Pixel im `numpy.ndarray`<br>
`image['maxgray'] - pixelGraustufenwert`<br>
berechnet und das Ergebnis wiederum als `numpy.ndarray` gleicher Größe zurückgegeben.

Es sind keine weiteren Imports nötig, das diese aus der Lösung von 8.1 übernommen werden.

In [ ]:
# Loesung 8.2.1 Invertierung

################################################################################
# Invertieren eines PGM-Bildes
# Erstellen eines neue Dictionary und Initialisieren mit den Metadaten eines 
# übergebenen Dictionary mit den Daten einer PGM Bilddatei entsprechend dem 
# folgenden Format. 
# image:    image['magic']: Magic der PGM-Datei
#           image['width']: Breite des Bildes
#           image['height']: Höhe des Bildes
#           image['maxgray'] Maximal vorkommender Grauwert.
#           image['pixels']: numpy.ndarray mit zeilenweisen Graustufen des Bildes
# return:   Neues Dictionary. 
#           Alle Metadaten des Parameter image werden übernommen. 
#           image['pixels']: 
################################################################################
def invertImage(image):
    result = dict();
### BEGIN SOLUTION
    result['magic']   = image['magic']           # Das Magic aus der Datei ("P2").
    result['width']   = image['width']           # Breite des Bildes in Pixeln.
    result['height']  = image['height']          # Höhe des Bildes in Pixeln.
    result['maxgray'] = image['maxgray']         # Maximal vorkommender Grauwert.
    
    #
    # Invertieren: 
    # max.Grauwert - 2-dim-numpy-Array-Objekt
    # Das ist alles!
    #
    result['pixels']  = image['maxgray'] - image['pixels'] 
    
### END SOLUTION
    return result


###########################################################
# Test
###########################################################
try:
### BEGIN SOLUTION
    infile="./figures/a7_dreifach.pgm"
    outfile="inverted.out.pgm"

    image = readPgmImageFile(infile)
    inverted = invertImage(image)
    writePgmImageFile(inverted, outfile)

    with Image.open(outfile) as i:
	    display(i)

except ValueError as e:
    sys.stderr.write(str(e))
except OSError as e:
    sys.stderr.write(str(e))
except Exception as e:
    sys.stderr.write("Unerwarteter Fehler:")
    sys.stderr.write(str(e))
    raise e
### END SOLUTION
    

### 8.2.2 Schwellwertoperation
Die Schwellwertoperation, alle Graustufenwerte unterhalb eines Schwellwert-Graustufenwertes auf `0` und alle gleich oder darüber auf den maximalen Graustufenwert ist "etwas" komplizierter - ein Zweizeiler:
```
# Kopie des Orinalbildes
result['pixels']  = image['pixels'].copy()# Die Pixel des Bildes als Kopie.
    
# Pixel nach Schwellwert ändern
result['pixels'][result['pixels'] < threshold] = 0
result['pixels'][result['pixels'] >= threshold] = image['maxgray']
```

Vor dem "Zweizeiler muss im Ergebnis-*Directory* eine Kopie des vorhanden Graustufen-Arrays erzeugt werden. Die Graustufen im so erstellte, fertige Array werden im Anschluss geändert:

`result['pixels']` gibt das kopierte `numpy.ndarray` zurück, dessen Pixel im Anschluss geändert werden müssen.

Der Zugriff auf ein einzelnes Pixel erfolgt "normalerweise" mit `result['pixels'][y][x]`, wobei y die Liste mit Pixeln für eine Zeile und x das Pixel der Spalte in der Zeile auswählt.<br>
Statt dessen wird die Operation im ersten Fall `result['pixels'][result['pixels'] < threshold]` auf alle Pixel im 2-dimensionalen Array ausgeführt, deren Graustufenwert kleiner als der verwendete Schwellwert sind. Alle diese Pixel, egal wo sie sich in Array befinden, werden durch `0` ersetzt.<br>
Die Änderung der Pixel, deren Graustufenwert größer oder gleich dem Schwellwert sind, erfolgt analog.

In [ ]:
# Loesung 8.2.2 Schwellwert

################################################################################
# Schwellwertbearbeitung eines PGM-Bildes
# Erstellen eines neue Dictionary und Initialisieren mit den Metadaten eines 
# übergebenen Dictionary mit den Daten einer PGM Bilddatei entsprechend dem 
# folgenden Format. 
# image:    image['magic']: Magic der PGM-Datei
#           image['width']: Breite des Bildes
#           image['height']: Höhe des Bildes
#           image['maxgray'] Maximal vorkommender Grauwert.
#           image['pixels']: Liste von Zeilen mit Liste von Pixeln in einer Zeile
# return:   Neues Dictionary. 
#           Alle Metadaten des Parameter image werden übernommen. 
#           Alle result['pixels'][y][x] werden berechnet aus 
#           image['pixels'][y][x] < threshold ? 0 : image['maxgray']
################################################################################
def thresholdImage(image, threshold):
    result = dict();
### BEGIN SOLUTION
    result['magic']   = image['magic']        # Das Magic aus der Datei ("P2").
    result['width']   = image['width']        # Breite des Bildes in Pixeln.
    result['height']  = image['height']       # Höhe des Bildes in Pixeln.
    result['maxgray'] = image['maxgray']      # Maximal vorkommender Grauwert.
    
    #
    # Invertieren: HIER
    #
    
    # Kopie des Orinalbildes
    result['pixels']  = image['pixels'].copy()# Die Pixel des Bildes als Kopie.
    
    # Pixel nach Schwellwert ändern
    result['pixels'][result['pixels'] < threshold] = 0
    result['pixels'][result['pixels'] >= threshold] = image['maxgray']
    
### END SOLUTION
    return result


###########################################################
# Test
###########################################################
try:
### BEGIN SOLUTION
    infile="./figures/a7_dreifach.pgm"
    outfile="threshold.out.pgm"

    image = readPgmImageFile(infile)
    thresholded = thresholdImage(image, 150)
    writePgmImageFile(thresholded, outfile)

    with Image.open(outfile) as i:
	    display(i)

except ValueError as e:
    sys.stderr.write(str(e))
except OSError as e:
    sys.stderr.write(str(e))
except Exception as e:
    sys.stderr.write("Unerwarteter Fehler:")
    sys.stderr.write(str(e))
    raise e
### END SOLUTION
    

### 8.2.3 Glättung
Bei der Glättung mussten Sie für jedes Pixel des Zielbildes den Mittelwert der Graustufenwerte des betreffenden Pixels und seiner 8 umgebendem Pixel im Quellbild berechnen.
<center>

![Glättung](./figures/a7_7_2_3_1_glaettung.png)<br>
Abbildung 7.2.3.1.: Glättung durch Mittelwertbildung über 9 Bildpunkte
</center>
Sie erinnern sich vielleicht, dass Sie für die Berechnung das Quellbild verwenden mussten, weil die Graustufenwerte im Pixel im Zielbild durch den Vorgang des Glättens fortlaufend verändert werden.

Auch für diesen Vorgang gibt es eine fertige Funktion:
```
from scipy.ndimage import uniform_filter
...
result['pixels'] = uniform_filter(image['pixels'], size=3, mode="nearest")
```
Wieder wird das gesamte Array des Zielbildes von der Funktion zurückgegeben.<br>
`uniform_filter` übernimmt
* mit `image['pixels']` das Array des Quellbildes, 
* mit `size=3` die Größe der Matrix, über die gemittelt werden soll und stellt
* mit `mode="constant"` wird festgelegt, dass die Pixel außerhalb der Grenzen der Pixel, deren Graustufenwerte im Rahmen der Größe der Matrix gemittelt werden, mit einem konstanten Wert belegt werden. 
* Der Wert, mit dem die Pixel außerhalb der Grenzen der Mittlung belegt werden, wird durch den Parameter `cval` bestimmt. Die Voreinstellung ist `0`, also schwarz und hat damit ohne explizite Angabe schon den richtigen Wert für den schwarzen Rahmen, den die Aufgabe fordert.

In [ ]:
# Loesung 8.2.3 Glaettung
from scipy.ndimage import uniform_filter
################################################################################
# Glätten eines PGM Image Dictionaries
# image:    image['magic']: Magic der PGM-Datei
#           image['width']: Breite des Bildes
#           image['height']: Höhe des Bildes
#           image['maxgray'] Maximal vorkommender Grauwert.
#           image['pixels']: Liste von Zeilen mit Liste von Pixeln in einer Zeile
# return: Geglättete Version des übergebenen PGM Images in einem neuen Dictionary
################################################################################
def smoothedImage(image):
    result = dict();
### BEGIN SOLUTION

# Die Lösung ist wegen sich wiederholenden Codes nicht optimal.
    result['magic']   = image['magic']        # Das Magic aus der Datei ("P2").
    result['width']   = image['width']        # Breite des Bildes in Pixeln.
    result['height']  = image['height']       # Höhe des Bildes in Pixeln.
    result['maxgray'] = image['maxgray']      # Maximal vorkommender Grauwert.
    
    #
    # Glätten 
    #
    result['pixels']  = uniform_filter(image['pixels'], size=3, mode="constant")
    
### END SOLUTION
    return result


###########################################################
# Test
###########################################################
try:
### BEGIN SOLUTION
    infile="./figures/a7_dreifach.pgm"
    outfile="smoothed.out.pgm"

    image = readPgmImageFile(infile)
    smoothed = smoothedImage(image)
    writePgmImageFile(smoothed, outfile)

    with Image.open(outfile) as i:
	    display(i)

except ValueError as e:
    sys.stderr.write(str(e))
except OSError as e:
    sys.stderr.write(str(e))
except Exception as e:
    sys.stderr.write("Unerwarteter Fehler:")
    sys.stderr.write(str(e))
    raise e
### END SOLUTION
    

### 8.2.4 Kantenerkennung
Die Kantenerkennung war die komplizierteste Funktion der Bildbearbeitung in Aufgabe 7.
<center>

![Kantenerkennung](./figures/a7_liste_7_2_4_1_kantenerkennung.png) !<br>
Abbildung 7.2.4.1.: Kantenerkennung durch gewichtete Mittelwertbildung über 9 Bildpunkte
</center>

Wie bei der Glättung wurden alle Pixel auf den Mittelwert der Graustufenwerte des Pixels selbst und seiner 8 umgebenden Pixel gesetzt. Zuvor wurden die Graustufenwerte aber mit den Werten einer Gewichtungsmatrix multipliziert. Da das Ergebnis negativ sein konnte, wurde für jeden Graustufenwert der Betrag (`abs()`) des Ergebnisses berechnet. Für ein Pixel an der gegebenen Position `y` und `x` sah das etwa so aus:
```
pixel = 0
# Anfang (1)
for dy in [-1, 0, 1]:
    for dx in [-1, 0, 1]:
        pixel += image['pixels'][y+dy][x+dx] * weights[w_y][w_x]
# Ende (1)
newValue = abs(int(pixel/9))
```

In dem Code ist ein Block mit (1) markiert. In diesem Block wird für ein Pixel die Summe des Grauwerts des Pixels und seiner 8 umgebenden Pixel berechnet, die zuvor mit dem Gewichtungsfaktor aus der Gewichtungsmatrix multipliziert wurde.<br>
Diese Funktion, angewandt auf die Graustufenwerte aller Pixel (außer den äußersten Pixel des Bildes, auf die die Matrix nicht angewandt werden kann) wird *Faltung* genannt. Diese Funktion und dieser Begriff werden Module höherer Semester noch einmal aufgreifen.

#### 8.2.4.1 Faltung
Für die *Faltung* steht eine eigene Funktion zur Verfügung:
```
from scipy.signal import convolve2d
...

# Gewichtungsmatrix
weights = np.array([
        [0, -1, 0], 
        [-1, 4 ,-1], 
        [0, -1, 0]], 
        dtype=np.float64)
    
# Zweidimensionales numpy.ndarray mit den Graustufen, konvertiert von int-Werten in 32-Bit-Float Werte
img_f = image['pixels'].astype(np.float32)

# Nur Faltung - KEIN Mittelwert (!!!!)
convolved = convolve2d(img_f, weights, mode='same', boundary='fill', fillvalue=0)
```
* Die Funktion convolved verwendet für die Berechnung den gegebenen Datentyp, der beim Einlesen auf<br>
`dtype=np.uint8` gesetzt wurde. Dieser Datentyp ist für die anstehenden Berechnungen ungeeignet und wird durch einen 32-Bit-Float Wert ersetzt. Ein 64-Bit-Float-Wert wäre ebenso möglich.<br>
Im Ergebnis-Array `convolved` liegen die Graustufen im gleichen Datenformat vor.

Um die Kanten in einem Bild erkennbar zu machen, reicht diese Operation aus.<br>
Allerdings werden viele Graustufen den erforderlichen Wertebereich von `0` bis `255` sowohl unter- als aus überschreiten. Außerdem ist der Datentyp `np.float32` nicht für das Schreiben in eine Bilddatei geeignet. Wenn Sie das Bild in diesem Zustand zur Ausgabe zurück geben wollen, müssen Sie zuvor noch folgende Funktion ausführen:
```
result['pixels'] = np.clip(convolved, 0, 255).astype(np.uint8)
```
* Die Funktion `np.clip(convolved, 0, 255)` begrenzt die Werte im übergebenen Array auf den unteren Grenzwert `0` und den oberen Grenzwert `255`. Werte die darüber und darunter liegen, werden durch den Grenzwert ersetzt. Das Ergebnis ist wieder ein Array mit Werten mit dem Datentyp `np.float32`.
* Die auf das zurück gegebene Array angewandte Funktion `.astype(np.uint8)` konvertiert alle Werte im Array in den Datentyp `np.uint8`, dem vorzeichenlosen 8-Bit-Ganzzahltyp, der für das Schreiben auf in eine Datei benötigt wird.

#### 8.2.4.2 Mittelwertbildung
Nach der Faltung entspricht das Ergebnisbild noch nicht ganz dem Ergebnisbild der Kantenerkennung aus Aufgabe 7.<br>
* Um den Mittelwert zu bestimmen, müssen die Graustufenwerte aller Pixel in der Ergebnismatrix durch $9$ dividiert werden.
* Um negative Werte zu vermeiden, müssen diese durch Betragsbildung positiv werden.
* Für die Rückgabe im korrekten Format müssen auch hier die Graustufenwerte in das vorzeichenlosen 8-Bit-Format konvertiert werden.

Die Zeile, die alle Aufgaben auf einmal erledigt, sieht so aus:
```
result['pixels'] = np.abs(np.trunc(convolved / 9)).astype(np.uint8)
```
* `convolved / 9` dividiert zunächst alle Werte der gefalteten Werte im Ergebnis-Array durch `9`, um den Mittelwert zu bestimmen. Das Datenformat ist hier noch `np.float32`.
* np.trunc(..) führt auf alle Werte im Array anschließend die `truncate`-Funktion aus, die Bruchanteile aller Werte "abschneitet". Das Ergebnis sind ganze Zahlen, immer noch im `np.float32`-Format. Negative Werte sind immer noch negativ.
*  np.abs(...) führt als nächstes die Betragsfunktion auf alle Graustufenwerte im Array aus, so dass das Array nur noch positive Werte enthält. Immer noch im `np.float32`-Format.
* .astype(np.uint8) konvertiert die Werte im Array abschließend in das für die Rückgabe erforderliche vorzeichenlose 8-Bit-Format.

**<u>Wichtig</u>**<br>
Durch die Wahl der Gewichtuntsfaktoren in der Gewichtungsmatrix in Aufgabe 7 wurde sicherstellt, dass die Werte nach der Glättung wieder im Wertebereich zwischen $0$ und $255$ liegen. Nur deshalb ist hier nicht die Verwendung der Funktion `np.clip(...)` erforderlich!

Nach der Mittelwertbildung sieht das Ergebnisbild genau so aus, wie nach der Kantenerkennung von Aufgabe 7.


In [ ]:
# Loesung 8.2.4 Kantenerkennung
from scipy.signal import convolve2d

################################################################################
# Kantenerkennung eines PGM Image Dictionaries
# image:    image['magic']: Magic der PGM-Datei
#           image['width']: Breite des Bildes
#           image['height']: Höhe des Bildes
#           image['maxgray'] Maximal vorkommender Grauwert.
#           image['pixels']: Liste von Zeilen mit Liste von Pixeln in einer Zeile
# return: Version des übergebenen PGM Images mit Kantenerkennung
################################################################################
def edgetImage(image):
    result = dict();
    
# Kopieren der Metadaten
    result['magic']   = image['magic']        # Das Magic aus der Datei ("P2").
    result['width']   = image['width']        # Breite des Bildes in Pixeln.
    result['height']  = image['height']       # Höhe des Bildes in Pixeln.
    result['maxgray'] = image['maxgray']      # Maximal vorkommender Grauwert.
    
    # Gewichtungsmatrix
    weights = np.array([
        [0, -1, 0], 
        [-1, 4 ,-1], 
        [0, -1, 0]], 
        dtype=np.float64)
    
    img_f = image['pixels'].astype(np.float32)
    
    # Nur Faltung - KEIN Mittelwert (!!!!)
    # Dafür nicht berechenbare (Randpixel) automatisch auf 0 setzen.
    convolved = convolve2d(img_f, weights, mode='same', boundary='fill', fillvalue=0)
    
    # Mittelwert nachträglich berechnen
    result['pixels'] = np.abs(np.trunc(convolved / 9)).astype(np.uint8)
    
    # Alternative: Bild nur mit Faltung behalten und auf uint8 "stutzen"
    # result['pixels'] = np.clip(convolved, 0, 255).astype(np.uint8)

### END SOLUTION
    return result


###########################################################
# Test
###########################################################
try:
### BEGIN SOLUTION
    infile="./figures/a7_dreifach.pgm"
    outfile="edged.out.pgm"

    image = readPgmImageFile(infile)
    
    edged = edgetImage(image)
    writePgmImageFile(edged, outfile)

    with Image.open(outfile) as i:
	    display(i)
except ValueError as e:
    sys.stderr.write(str(e))
except OSError as e:
    sys.stderr.write(str(e))
except Exception as e:
    sys.stderr.write("Unerwarteter Fehler:")
    sys.stderr.write(str(e))
    raise e
### END SOLUTION
    